In [33]:
# Install required libraries
# You can install these packages using pip if not installed:
# !pip install pandas sqlite3 pandasql

import sqlite3
import pandas as pd

# Set working directory
import os
os.chdir("/Users/jade/Desktop/Humana/Training")

# Load datasets using pandas
members_data = pd.read_csv("mdata.csv")
pharmacy_data = pd.read_csv("PU.csv")
social_data = pd.read_csv("Socialh.csv")
web_data = pd.read_csv("WA.csv")

In [35]:
import re
def convert_tenure(value):
    # Check if value is a string before processing
    if isinstance(value, str):
        # If the format is 'X - Y YEARS', calculate the midpoint
        if '-' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 2:  # Ensure there are two numbers for the range
                return (float(numbers[0]) + float(numbers[1])) / 2
        # If the format is 'X+ YEARS', take the number before the '+'
        elif '+' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:  # Ensure there's at least one number
                return float(numbers[0])
        # If there's just 'X YEARS', return the number
        elif 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:
                return float(numbers[0])
        # Default case for strings that don't match the expected format
        return None
    # If value is already a float or an int, return it as is
    elif isinstance(value, (float, int)):
        return value
    # Default return for unexpected types (e.g., NaN, None)
    return None

# Applying the function to the column
members_data['tenure_band'] = members_data['tenure_band'].apply(convert_tenure)
members_data['disabled_ind'] = members_data['disabled_ind'].replace({'Y': 1, 'N': 0})
members_data['dual_eligible_ind'] = members_data['dual_eligible_ind'].replace({'Y': 1, 'N': 0})
members_data['lis_ind']= members_data['lis_ind'].replace({'Y': 1, 'N': 0})

In [60]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np

# Function to remove high correlation and low variance columns
def clean_data(df, variance_threshold=0.01, correlation_threshold=0.9):
    # Step 1: Remove variables with low variance
    selector = VarianceThreshold(threshold=variance_threshold)
    df_reduced = selector.fit_transform(df)
    
    # Convert back to DataFrame to handle column names
    columns_kept = df.columns[selector.get_support()]
    df_filtered = pd.DataFrame(df_reduced, columns=columns_kept)

    # Step 2: Remove highly correlated variables
    corr_matrix = df_filtered.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
    
    # Drop highly correlated columns
    df_cleaned = df_filtered.drop(columns=to_drop)
    
    return df_cleaned

# Assuming your data is already loaded in these variables
# Apply the cleaning process to each dataset
members_data_cleaned = clean_data(members_data)
pharmacy_data_cleaned = clean_data(pharmacy_data)
social_data_cleaned = clean_data(social_data)
web_data_cleaned = clean_data(web_data)

In [61]:
print(members_data_cleaned)

         consec_tenure_month  dual_eligible_ind  disabled_ind  lis_ind  \
0                        6.0                0.0           0.0      0.0   
1                       59.0                0.0           1.0      0.0   
2                      119.0                0.0           0.0      0.0   
3                       24.0                0.0           1.0      0.0   
4                       60.0                0.0           0.0      0.0   
...                      ...                ...           ...      ...   
1527899                  4.0                0.0           0.0      0.0   
1527900                 23.0                0.0           0.0      0.0   
1527901                  3.0                0.0           0.0      0.0   
1527902                 24.0                0.0           1.0      0.0   
1527903                 12.0                0.0           0.0      0.0   

                id  
0        1551235.0  
1        1643841.0  
2         540296.0  
3         729600.0  
4     